Performing this in a notebook due to tensorflow weirdly not detecting GPU outside of an IPython environment.

In [5]:
!export LD_LIBRARY_PATH=/usr/local/cuda/lib64:$LD_LIBRARY_PATH

In [16]:
from fer import FER
import cv2
import pprint
import tensorflow as tf
import numpy as np
import pandas as pd

In [52]:
from pathlib import Path
from typing import List
from tqdm import tqdm

In [6]:
gpus = tf.config.experimental.list_physical_devices('GPU')

Emotion example:
```
{'angry': 0.0,
 'disgust': 0.0,
 'fear': 0.0,
 'happy': 0.99,
 'sad': 0.0,
 'surprise': 0.0,
 'neutral': 0.0}
```

In [33]:
detector.top_emotion(img)

('happy', 0.99)

In [37]:
PROCESSED_DATA_BASE_PATH = Path('../data/processed')
ATTRIBUTE_CSV_PATH =  PROCESSED_DATA_BASE_PATH / 'attributes.csv'
IMAGE_BASE_PATH = PROCESSED_DATA_BASE_PATH / 'images'

In [42]:
attributes = pd.read_csv(ATTRIBUTE_CSV_PATH)
filenames = attributes['filename']

In [61]:
detector = FER(mtcnn=True, )

In [62]:
results: List = []

for i, fn in tqdm(filenames.items(), total=248):
    img = cv2.imread(str(IMAGE_BASE_PATH / fn))
    top_pred, top_pred_confidence = detector.top_emotion(img)
    
    full_results = detector.detect_emotions(img)
    if len(full_results) > 0 and 'emotions' in full_results[0]:
        full_results = list(full_results[0]['emotions'].values())
    
    results.append((fn, top_pred, top_pred_confidence, *full_results))

  0%|                                                   | 0/248 [00:00<?, ?it/s]

100%|█████████████████████████████████████████| 248/248 [01:49<00:00,  2.26it/s]


In [64]:
results_df = pd.DataFrame(
    results, columns=('filename', 'top_emotion', 'top_conf', 'angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral'))

In [67]:
results_df.iloc[10:25]

,filename,top_emotion,top_conf,angry,disgust,fear,happy,sad,surprise,neutral
10,5_4.jpg,happy,0.95,0.02,0.0,0.00,0.95,0.03,0.00,0.00
11,6_0.jpg,happy,0.80,0.02,0.0,0.02,0.80,0.16,0.00,0.01
12,6_1.jpg,happy,0.68,0.07,0.0,0.06,0.68,0.15,0.01,0.02
13,6_2.jpg,happy,0.94,0.00,0.0,0.00,0.94,0.00,0.00,0.06
14,7_0.jpg,happy,0.94,0.01,0.0,0.01,0.94,0.02,0.02,0.01
15,8_0.jpg,happy,0.96,0.00,0.0,0.00,0.96,0.00,0.00,0.03
16,9_0.jpg,neutral,0.44,0.07,0.0,0.02,0.42,0.06,0.00,0.44
17,9_1.jpg,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,10_0.jpg,happy,0.99,0.00,0.0,0.00,0.99,0.00,0.00,0.01
19,10_1.jpg,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
results_df.to_csv('../results/fer_2013_results.csv')